In [139]:
import numpy as np
from numpy.linalg import norm
from scipy.sparse import diags, csc_matrix
from time import time

In [140]:
def getMatrix(filename, isDiag):
    with open(filename) as f:
        shape = int(f.readline())
        matrix = [[float(num) for num in line.split()]
                  for _, line in zip(range(shape), f)]
        if isDiag:
            matrix[0].insert(0, 0)
            matrix[-1].append(0)
            a, b, c = zip(*matrix)
            matrix = diags([a[1:], b, c[:-1]], [-1, 0, 1])
            matrix = csc_matrix(matrix)
        else:
            matrix = csc_matrix(matrix)
        b = np.array([float(num) for num in f.readline().split()])
        return matrix, b

In [141]:
def biCGStabSolve(matrix, b, eps, shape, x0, k):    
    r0 = b - matrix @ x0
    x0 = x0
    r2 = r0
    rho0 = 1
    alpha0 = 1
    omega0 = 1
    v0 = np.array([0] * shape)
    p0 = np.array([0] * shape)
    while True:
        rho = r2 @ r0
        beta = (rho * alpha0) / (rho0 * omega0)
        p = r0 + beta * (p0 - omega0 * v0)
        v = matrix @ p
        alpha = rho / (r2 @ v)
        s = r0 - alpha * v
        t = matrix @ s
        omega = (t @ s) / (t @ t)
        x = x0 + omega * s + alpha * p
        r = s - omega * t

        k += 1
        if norm(r) < eps:
            break
        r0 = r
        rho0 = rho
        alpha0 = alpha
        omega0 = omega
        v0 = v
        p0 = p
        x0 = x
    return x

In [142]:
def printSolution(matrix, b):
    eps = 1e-5
    shape = matrix.shape[0]
    x0 = np.array([0] * shape)
    k = 0
    
    start = time()
    x = biCGStabSolve(matrix, b, eps, shape, x0, k)
    end = time()
    start2 = time()
    x2 = np.linalg.solve(matrix.toarray(), b)
    end2 = time()
    print('My solve:\n')
    print(f'{x.round(5)}\n')
    print(f'EPS = {eps}\n')
    print(f'Shape = {shape}\n')
    print(f'Count of iterations = {k}\n')
    print(f'Mean = {np.mean(x)}\n')
    print(f'Time = {round(end - start, 5)} sec\n')
    print('\nNumPy solve:\n')
    print(f'{x2.round(5)}\n')
    print(f'Mean = {np.mean(x2)}\n')
    print(f'Time = {round(end2 - start2, 5)} sec\n')

In [143]:
matrix, b = getMatrix('test10', False)
printSolution(matrix, b)

My solve:

[-195.34591   26.53793  309.59768  -34.31888   47.3909  -126.5331
  214.97703  -20.73636  249.00413 -260.32933]

EPS = 1e-05

Shape = 10

Count of iterations = 0

Mean = 21.024408467227477

Time = 0.00067 sec


NumPy solve:

[-195.34591   26.53793  309.59768  -34.31888   47.3909  -126.5331
  214.97703  -20.73636  249.00413 -260.32933]

Mean = 21.024408466560754

Time = 5e-05 sec


In [144]:
matrix, b = getMatrix('test20', False)
printSolution(matrix, b)

My solve:

[-18.23957 -14.64113 -10.45047  33.47596  23.60765 -44.7088   45.67713
   8.19735  33.83973   8.87599 -25.66239  -9.31423 -19.90542 -23.84705
  34.99552  29.66388  22.22547 -12.89704  54.27331  46.53818]

EPS = 1e-05

Shape = 20

Count of iterations = 0

Mean = 8.08520327117203

Time = 0.00474 sec


NumPy solve:

[-18.23957 -14.64113 -10.45047  33.47596  23.60765 -44.7088   45.67713
   8.19735  33.83973   8.87599 -25.66239  -9.31423 -19.90542 -23.84705
  34.99552  29.66388  22.22547 -12.89705  54.27331  46.53818]

Mean = 8.085203088065231

Time = 9e-05 sec


In [145]:
matrix, b = getMatrix('test30', False)
printSolution(matrix, b)

My solve:

[ -53.64412  -35.47177 -118.89011  104.64771 -128.42506  249.20309
   70.78105   57.49387 -158.63259  -76.66191    6.34065  311.24475
   96.8618    29.04643  -64.97601  -95.87315   74.15824  234.61787
   80.88134   72.80343  -74.44886   58.00882 -148.87989  -63.78908
   99.56411 -122.365   -133.83062   86.21966  -89.91691   25.67745]

EPS = 1e-05

Shape = 30

Count of iterations = 0

Mean = 9.724839647431981

Time = 0.01143 sec


NumPy solve:

[ -53.64412  -35.47177 -118.89009  104.64771 -128.42506  249.20307
   70.78104   57.49385 -158.6326   -76.66189    6.34065  311.24474
   96.8618    29.04643  -64.976    -95.87315   74.15823  234.61787
   80.88134   72.80342  -74.44885   58.00882 -148.87987  -63.78908
   99.56411 -122.36499 -133.83061   86.21965  -89.9169    25.67744]

Mean = 9.724839707418928

Time = 0.00016 sec
